Suddenly, a scheduled job activates the system's disk defragmenter. Were the situation different, you might sit and watch it for a while, but today, you just don't have that kind of time. It's soaking up valuable system resources that are needed elsewhere, and so the only option is to help it finish its task as soon as possible.

The disk in question consists of a 128x128 grid; each square of the grid is either free or used. On this disk, the state of the grid is tracked by the bits in a sequence of knot hashes.

A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is free (0) or used (1).

The hash inputs are a key string (your puzzle input), a dash, and a number from 0 to 127 corresponding to the row. For example, if your key string were flqrgnkx, then the first row would be given by the bits of the knot hash of flqrgnkx-0, the second row from the bits of the knot hash of flqrgnkx-1, and so on until the last row, flqrgnkx-127.

The output of a knot hash is traditionally represented by 32 hexadecimal digits; each of these digits correspond to 4 bits, for a total of 4 * 32 = 128 bits. To convert to bits, turn each hexadecimal digit to its equivalent binary value, high-bit first: 0 becomes 0000, 1 becomes 0001, e becomes 1110, f becomes 1111, and so on; a hash that begins with a0c2017... in hexadecimal would begin with 10100000110000100000000101110000... in binary.

Continuing this process, the first 8 rows and columns for key flqrgnkx appear as follows, using # to denote used squares, and . to denote free ones:

##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
|      |   
V      V   
In this example, 8108 squares are used across the entire 128x128 grid.

Given your actual key string, how many squares are used?

In [163]:
greatly = []
rowlen = len(inpt[0])
for ln in inpt:
    greatly.extend(ln)

In [398]:
from collections import defaultdict

class OneDimMask(object):
    def __init__(self, inpt):
        self.onedim = []
        self.dimlen = len(inpt[0])
        
        for dim in inpt:
            self.onedim.extend([int(x) for x in dim])
        
    def north(self, point):
        if point < self.dimlen:
            return None
        offset = point - self.dimlen
        if self.onedim[offset]:
            return offset
        return None
    
    def east(self, point):
        if point % self.dimlen == self.dimlen-1:
            return None
        if self.onedim[point+1]:
            return point+1
        return None
    
    def south(self, point):
        if point+self.dimlen < len(self.onedim):
            if self.onedim[point+self.dimlen]:
                return point+self.dimlen
        return None
    
    def west(self, point):
        if not point % self.dimlen == 0:
            if self.onedim[point-1]:
                return point-1
    
    def find_neighbors(self, point):
        north = self.north(point)
        east = self.east(point)
        south = self.south(point)
        west = self.west(point)
        return [x for x in (north, east, south, west) if not x is None]
    
    def group(self):
        groups = {}
        for i in range(len(self.onedim)):
            if self.onedim[i]:
                c = Node.nodeFactory(i)
                c.contacts = [Node.nodeFactory(x) for x in self.find_neighbors(i)]
                groups[c] = c.contacts
        return groups
    
    
# class Node(object):
#     def __init__(self, id):
#         self._id = id
#         self.neighbors = []
        
class Node(object):
    nodes = {}
    
    @classmethod
    def nodeFactory(cls, _id):
        if _id in cls.nodes:
            return cls.nodes[_id]
        else:
            n = Node(_id)
            cls.nodes[_id] = n
            return n
        
    @staticmethod
    def parse_input(blob):
        for line in blob.split('\n'):
            line = line.strip()
            if line:
                try:
                    _id, cts = line.split('<->')
                except:
                    print("line:", line)
                n = Node.nodeFactory(_id.strip())
                c_ids = [x.strip() for x in cts.split(',')]
                n.contacts = [Node.nodeFactory(x) for x in c_ids]
        
    def __init__(self, _id):
        self.id = _id
        self.contacts = []
        
    def __hash__(self):
        return self.id
    
    def __repr__(self):
        return "{} <-> {}".format(self.id, [x.id for x in self.contacts])
        
    def all_contacts(self, acc=None):
        if not acc:
            acc = defaultdict(bool)
        acc[self] = True
        
        for cont in self.contacts:
            if not cont in acc:
                cont.all_contacts(acc)
            acc[cont] = True
        return acc
        
        

In [447]:
mask = OneDimMask(inpt)

In [448]:
groups = mask.group()
type(groups)

dict

In [449]:
zero = Node.nodeFactory(8)
zero.all_contacts()

defaultdict(bool,
            {8 <-> [9, 136]: True,
             9 <-> []: True,
             136 <-> [135]: True,
             135 <-> [136]: True})

In [450]:
master = []
for ind in groups:    
    if groups[ind] != 'x':
        ac = ind.all_contacts()
        master.append(list(ac.keys()))
        for a in ac:
            groups[a] = 'x'
    

In [451]:
len(master)

1074

In [219]:


class Node(object):
    nodes = {}
    
    @classmethod
    def nodeFactory(cls, _id):
        if _id in cls.nodes:
            return cls.nodes[_id]
        else:
            n = Node(_id)
            cls.nodes[_id] = n
            return n
        
    @staticmethod
    def parse_input(blob):
        for line in blob.split('\n'):
            line = line.strip()
            if line:
                try:
                    _id, cts = line.split('<->')
                except:
                    print("line:", line)
                n = Node.nodeFactory(_id.strip())
                c_ids = [x.strip() for x in cts.split(',')]
                n.contacts = [Node.nodeFactory(x) for x in c_ids]
        
    def __init__(self, _id):
        self.id = _id
        self.contacts = []
        
    def __hash__(self, _id):
        return _id
    
    def __repr__(self):
        return "{} <-> {}".format(self.id, [x.id for x in self.contacts])
        
    def all_contacts(self, acc=None, depth=0):
        if not acc:
            acc = defaultdict(bool)
        acc[self.id] = True
        
        for cont in self.contacts:
            if not cont.id in acc:
                cont.all_contacts(acc)
            acc[cont.id] = True
        return acc


In [446]:
# Input for ljoxqyyw

inpt = ['11101100010010101001011001100101100011110000000111001000011001010110010111010111101101000011101111000111010000110010110011100011',
 '01110001100101101111100000110100001110100110100111110001011010101101010100011010001101111111111000001101001011000011011011110010',
 '10111110001011010101101001111011100011110100101110110011111111101101110100010101011110001011110100111110101011011100111101101010',
 '11001111101001000101001111001000001100011010100101000011010111001111100000001110111010110111000000100001110100110101110110110101',
 '11110011000111000111001100010101100110001100111011001111011011101100011100100100011110001011101111110010111011001001101011011010',
 '00000100010110110000101101011000000001011110111011010110110000011011110000001110010111101100001101101101111111011000010011001011',
 '01100010101111011110010100111011101011001010011100011100111000001110011000010101011111100000000111111001101110110110101101010011',
 '10101000000110100100100011110101110101111101101011100011111100011001000000001011011100001010101001110010111001011001100101011111',
 '01111100110110100111100111111111011100101110110010010110101011000100110110010110110100010101101100100001000011100011000011001010',
 '00000100010100101100001110100000011100110011000010111010111101010000100101000001101101000100101000100000111011001000110111001110',
 '10111001010001001001001011100100001100111001010110011100001001000001000011011000011011100011001000001110101010111111001101010111',
 '01110000010011010101101000101111001001000110001100111011011100100100111011100101001110000111101001101010101110110011010001001010',
 '01010100000001111001100111110111010010110110100000011111001110110110111000101110010011111110101010101111000000011010111111011110',
 '01010101101101101111000111011111110100010100001111110011111011111111110010000111110110111110010110111111111010010001111101001111',
 '10111101110100101011000111010110011010111100001001010110001001110100101011001100011010101110011011110110001011000100100001001000',
 '11101101000101011000111100011010010101000101111011111011011001100000111111100000010000111111110010110100110111001110110100101111',
 '01110101000111000000100111101000110111010100100000100010110110111100111011011001010000111011000111001101110001100101100101010011',
 '01101010101011101110110100000011011000000011100101110110100110111100000101101011110000010110101010001001011001111100000010110001',
 '00101001100111011010001001111010011011101001110011011110010010010011000000001101110110001110101010010000011010111110110000010001',
 '11111001011111011011001011011001110110000111011000010010001001010101010010010110001010110101010111111111111001001001010001000101',
 '11001011111011101010110110001000111100010000001101011111110100101100010101101101001000101000100101001100110011111000111101110000',
 '11110110110101110100011011011100100010111000111110101101010010011011110111100110100010101000101110001101111111100110000100010011',
 '11011010011111011011100000001111000001101001000100001011011010100101001111100000111001000000111111011010101110101001100001000110',
 '10101110000101001011111001110011011001100010011000001001111101110101001101100011100000100010100000010110110110001110000001111101',
 '01101100101101001101110111011001101100100110110100000000001001000110110111110000101000010101001000011100010101010101000001010000',
 '11110101010110011000001111011000100111101001101010000010111011001111011100100110111101000001111111111100111001001000101000110101',
 '01011011100110000001110011001001011111111001001111111011110010101110010011011010011100110110010000111010010010111101001101000000',
 '00110100010110011110111010010101111100100100100101000001100101010010010011000111100000111010000111110010011111101101111011101010',
 '11111010111011011101010101111110011111110011101011110011101110001001011101010010110010111110100110111101111011110111011001110001',
 '11101001110110001111110100000000110010001111011111000101110011110110010011011010101100011111101111100111010101000110000111011111',
 '10100000111011111101100100001110011010010011101101001101100110001111100001110100110000010111110101100011110011110110110111101110',
 '10110101000010111000100011101001100010011100011110101100000111100100001000101011001100011110100011100001011101101011010110110001',
 '11101110010001101101000011110001110100000010011001011000100000011010100111101001001101010001101110101100111001111101010001010111',
 '00001110100001111011011111100010011111011101110000101100100111000100010000010001011100011110101101101001101100111001001001001010',
 '10011000110101101000000100001000100110000100110101101101000110011110010101011101101001110010000010110111110111110001011100100110',
 '00010111111100001010100011000001111111011000101111101110111111101100111100011000000001011001010001101100101110110100010000111101',
 '00000011001000000100100010000001000110000000000111110101001011011100000110110001011110111011000111111110001011000010111101101100',
 '01001111011011101110001000110010000100000001001100001111011011010100100000101100111100100000100000101111111010001110000000101001',
 '10000101110011001011100111100101110110011101001011001010100010010111110001001101110100101111010010010001010011100110101011111111',
 '10001110010000011110011001010111001000001000111100000011110000001101010110110110011110011011110110001100110100001001110101110100',
 '01111111001011000101110100010000110101110101100001111110001010110000111011111111011110001100101100011111101110011011000110010001',
 '10011000110000101111001000101101001000101110000010100001011110010001011001001100010001011100111101100111100100010111001011001011',
 '00100001100101000101111001100110100011101111111010100001100111000111111111101001000001010101110111100100011010100111101011111010',
 '01010111011001111101101011001010111110001001011010100010000110010101010011100001111001000011001011000000111000101010111000011010',
 '10010000111111001110001101010111010110101111110001000001110110101100010111010011101101001000010100111011001111010101101110011111',
 '00001001101100100111100010010111111010100111101101000000110110110110111000101001011101101111111100000010001001000010000110010111',
 '11011101111100110010010110000101110010111000011111000111000000110101000010000010111101010001111010000111000010101100011101110101',
 '00110101110110100010111010100110100001000000001010000001000101010001010101011111100001001011001010101100010011011001110101110101',
 '10010100111100101110010000101011101111100111111000110011010000011011110111110011101011001111011110011110100001001111011111100011',
 '00110001000010101110110001001100100001010111011101011110001101010001101001011111010111001011010001110111101101001110001010001110',
 '01001101001000010110010000000100100111011110101000110111110110000001111010010001100010101110111100010001100111011101000000100010',
 '00110111111010011001001001110100001001001111000011101011110001011100000100100100000111011011100111010010101010011010110101010101',
 '10010001111111001000011100100000111101000110111110000000001110100100010001101010010010011010001101110100110011000110000111110110',
 '00011101100101010001001000100001011100011001010010001001100101010010011101010010001011001110011111111111110010001001010001011111',
 '11111000110010100001011101011010110110101100001100100011101111110010110110000110101001110011111100001100100101010110100100111001',
 '11010000101000000000101001110000100111001100001110000110010100010110010101010101110100011100101101100111111011011110011011000100',
 '01101011101101111110000000011101101100011000010000100001001011001001000111111100011100101000010111100010010100100110011001010101',
 '11010111001001110000000000000111100001010001001001101000001011000010110000010100010110110100110001000010011011110111100001011110',
 '00011101001000001101011111011000110100111110000111000011110011111110011011110010110010111011011001011110001110010001110100011111',
 '01111000001010100011101110001100101100010100000100010000111111011011111001001110010101111011011111110100001111100011100100011011',
 '00101101000111001101101001111001101011011011010000011111111001110000011010110101000111100000110011101000011110100000001001000010',
 '01001101111110001110111010001001101110010100001111011111110010101111001001111010111001001001110000100110101001100011010101111000',
 '10010110111011111110010110110000010010011010001000010111010011110110101101111011101100111001101111100111101001010100110010101001',
 '01001000111000001110010011011101101110001011100001001010011111100001100001111000011110000011010110111110111101010001111011011101',
 '10111100010000111001001111100001001001011001000001010111000010010000111010100111010010000111001010101110010100001100000011001011',
 '10011111011011100001100100010010011011110011110111011001000111110001111011010011010000010000110101000000000011001000011000100101',
 '01000101000111101011000001000100110101001101010100110000101101001010110100001111011011100000111011001011001101110000100000011100',
 '01001101111100110110100100000101000100000110010001100110010011011110011110100011010011100010001001000110010000110011110110011101',
 '01011101001111010010001000101010110011111000011010011100011101110000001010000111101101011011100110011111111010011100100111111100',
 '00101100010101011110001111011010111100001000000000101101000001001000111011111111110111100000001011000100111011110010010010111011',
 '01010100011001100010111110011110011110001100001110100001000011100111110101111001001101100111011101010100100100000111110101101011',
 '01100100100101001001011000001100010110101001101100000000010110110000100111110111101101110111111110011001111111001001110000111111',
 '01011100101001000010001001101010110111100000001110000110010000110011000111011101010001010001100011101101000010111110110000010011',
 '11001000111100000101111101000000100111011100110111100010101101010101010001111010101000010011011000111001010011110000001011101101',
 '10010110001000110100111100010101110100010100111011110010001010101111110111001000000101110010101111001001010001101101101111110101',
 '00101100000010100110101100111011111011100111001110110101110100110111101110001000001000100100111111000000111010101111100111000000',
 '11101101010100111100000110110011000111001101010000011010001100110001010101100010000011000001100111001000001000110011101110011111',
 '11010101110101101011001100111110101000000101011011110100110100011110110110011001010111010011111100101001111100011011011100100100',
 '10010110010100010100001110000100110101000000111111010101000111001100111010001101000100101011101010101011101100000000000011100010',
 '10101101111111100100001100100111111000101010110011011100101110110000101010000100010100111110100001101010011111010011011100001011',
 '01111100111010111000111000111101001001110010000000110001101000010100001000011001110000100111111111101011001000110111000111101100',
 '01110001010011000101101010111010110110001011000101011000100110111100111011000010001100011000100101001000100001010100110101000011',
 '00010100001111001100111100111010101111100011111010100111110000100111001101110001011111100101000110000110000101011111111101111001',
 '10111000000001011110111000001111010010011111111100111111010011111010010100011110101001100011100110010101001100010100110101010111',
 '11010111000100100101010011001100010001000100000101101001100010001001101101111000111010100100000100001100111000000001011100001010',
 '11001100101100000000000000111111100001000101001101100000010101000100101010111001111100100011010001101101100010101010100011011010',
 '01001001011111001010100100010010001010101010101001010000010011101111001111001011111111000111000100011101001110001011001100110011',
 '11111011000100111111100000011110001101000101111000010101111110001010010101010101111001101011001001011100001011100000010101011010',
 '00101011110001011001100010101010011101011110110100000010010100111001001010110011000011001101101111100101110100100010111111111011',
 '10100101111111000000000011010011100010010001110000010110110111000010101010010011110110011010101111100011101001000011001001101011',
 '01011100101011010010111100101010100111100000111101100000000000011011111111111011001001010110110010100101001100110001101010000101',
 '10010100011101111110101001001011010111000000001100011110101111010010001010011000110111110111100101011110000110101010110001001010',
 '11111101000010010000000000100111000111010000010101010010000101010001001101011110000001000001111000010000101100011101010110101111',
 '01011110110100000011001001101100000010001110110101110000100011100110100100010001100010000100001111111111110001001001010111010110',
 '01001001010010110000110000010100110100110111011100011100010100101001111010110101110101110011101001111011010000011010101010100110',
 '10100011110101001011011110110111100001001011100100101000111010000111111010110011101101000111110010001011001101111000111110111100',
 '11110011000111110101011010101101111011010100101001010001100010011000111010000101100001011110000110101111111100001110110010110100',
 '00111000101101101001001111001001101001001111000010110100111100010001011011011100000001001100010001101110110100100101010100100110',
 '11100100111110011110111110111001001101010010001001000110101010000010000101010111000101110101100100000111101000001101111011110011',
 '01011000111111101010011100110000000111101011010100011001011111111110010111001100001100111000000010100100011100100011100110001001',
 '00000010011101111000110111100110111111100110101000000110111010101011001010110000101111101111001011000000011010111110111101101100',
 '11110101101011001010010011011100101001110011001001111011000110011001101000100101111100010100001001101100000100000110110011001010',
 '10011000001110100110001000110010001001000110011100110001001100010111101100000110000001010100001001110011101000010100100000010001',
 '11100001111111001110110100101000110101000101000101101000101101000101010100010111010011110000110111001101110111001111111001101110',
 '11011001101001110110010010000100011110111110110110101000111010111101110000111100100101000001011000000110001111000110100001111011',
 '00111111000011110001010001010010010011001111111100011101100011010100001010101011101000011011110100110000000110010011011110111110',
 '11000101110100111001001100101100011011101100000000001001111000010100011001101100100100100101010101111010000111110010010101000010',
 '00001011110101010001111110001110101100001000010110011110000110001110110011111000111010000101001011101110111001101011111011100100',
 '11011010100011001111000011111100111000111001100010000100110001010110011000110101110101100000100110001100110110011110010001011101',
 '00100000110110010110101010110000100000101110000010101111101010011101011011010010001101111010110111010011001001101100100111100101',
 '01001111110110111101110111010010110100111101000101100010001001111001100011100000110111010000110111101110100101010010010100101010',
 '10111111011010100000101011110101000011011110000100000000010111000101100101101001011001001001001010011100101100100001001001100000',
 '10001001000010110011010101101101001001011110010111101010001011001011011010110111101000011101101111001101000110101110101110011011',
 '00100001001011000110011110100001110110110101001101100010111010000110100010111011001010001001111001101101110010011111000111111001',
 '11011010110100110111011111110110110001011001010100101011000010110010001100111101010010001110011010111111000111101000110001100101',
 '00101100001101011001000100111111011101100100110101100000110001101100101010111010101111110011110101101100011001011100100100011000',
 '11000010111110011101101101111010110111011100101011000111110000111100011100011010011111110001101001011011110110101110100101011001',
 '11010100001011111001010110111110001000101010011111010000101001010001111101000100100101101101011100111111110000110001001111010101',
 '11100001100010101010100111010000100000001111011000101010000111010000000011001110000111101100001101111100101110000100110111001001',
 '10010001100001100011010001001111111111011010011001110110010000011001101110110010110110100011100001101001011010011110110100100110',
 '00011001110111000101001000000110100100010000111000110010100011011100001000101001011000110000110001111010001111011010000111010011',
 '10010100100101101011110100000110000101011010000110100010001011100011011100111111011011010001100111100100110010111111010000100110',
 '10100001010101011100011011000000100001100101011010111100111001001011100010100001110010001110110011111100011000111000100001010000',
 '00100011011011111011011111011111101001001011110100111010111101101001000011011000111110110110010111100000101101001110110110011110',
 '00101101110011111101100001111100000011100110100100011000011110110001100010101010011111000101101100000010011001101100110101111110',
 '10111001000100110110101011011101000010011110000011111010111000101110000000011110001111001000011101000001100000101001001111111001',
 '11010110111110111010100011001010010000001000000001111001001111011111011100110010001011101000110010001000000111000100001101111011',
 '00110011011001000111101001011000010001110001001010101101000110000011110011101110111110010011001010001010111110111010010101011000']

In [349]:
# input for flqrgnkx

inpt = ['11010100111101110110101111011100101111111000001110001111100001000001011011001100111110101000101111000110110100011111100111100110',
 '01010101111010101011001111000100111110111111111011011110000101101101110011101100001011000110011011011101101000100110010001100100',
 '00001010110111110001001111111010010000001110100011101010100000010101001101110110011101110110101011110011101101111011001000110001',
 '10101101001111011010001010001100110101111011100011111011100110010111010000101100000011100110001101100111001011001010111101100010',
 '01101000001011111110010010001100010101011000011101101010101010101010000100011101111100100110001101001111100101101101001100011010',
 '11001001111101011111111111111100010001100100101001010110010111111001111010010100111100110010001110101000101010101010100101001011',
 '01000100110000000001101001110010011000100110110011101110000011100011101111000000111010001001100010001101000100110111010001100011',
 '11010110011100100011011000001000001100001100011000111001010010010100011010101100110110010011011101110110010111100000011000110110',
 '11110111111001110010100101011111110010111010110011010011000000010011101101011100100111101001010011000011100000010110111110010011',
 '11010111100000011000111000001111001110110011100010111101000000101011100110100010100101011000001011111001100101111001100010010001',
 '00010000101100100101000101100001110111101101101001110110001101100100010001110001000111100010100101001000101010010010011100010010',
 '01111111111100001001000101111001111010001110100101101111011001110100110110011110011000000110001011000100001000100111001100101010',
 '00010110001001110101101011110001000000010100001001000101011111000101010100011101100010111110001101010110000111100001101110010011',
 '01001100111001100010111011100111000000110110100100010101000011111110100110110011110111011100011101001101101001110100110011110101',
 '10001000011011001011001010110110000010100100010001110110101111001111110101010101110101101110110111000101000011100011101000000110',
 '11001101101101001110100110100001010100011001001001000111000110011111100000001100101001001111011011100000001110000011111011101100',
 '00110110011111100011000001111000110101011100010111110011010111001011101001111100010101000011100100010100000100011000001110010010',
 '11111101001010100101000101100000110100010100010010010010101011100111010000001100101010001010001000011010110000100100011010100011',
 '01111110101011111000011101111100101111110010001000010111110111101100000101011010101101001111001010111111100101111010101000100001',
 '10101100010001100011100100011100010000100110111100100000110101011100100011011011011111000100011010010111000100101110100101010010',
 '00001011000011101000011110010111110100101011011011001001111100111111001111111000001101000100000010101111100100010001001100011001',
 '10010000101011110001000000100110101100010100111010101011100000111100001111111110110110000100110110111000000101000110000010111010',
 '00110111100101111001111100111100011101100000101011101110000110001110110010010101101010000001010001001101001110000111111001000111',
 '01010010011010100110101010000001111010011011010000001001000001000110011001001000101010101001110011111111100000110100100010101111',
 '00000010010101011110101001001101110100110001110000001010101101111010111110111001000110110010110111011001100110101010100001001001',
 '00100100111101001000010110010001001000100111011010111110000101111100011011111001011010010110111100000001011000000000001101100010',
 '00001010110101000000011100001101110100000001010011110010100001011001010100010101101000101100010001000000000011100010001011101101',
 '11100100000001010101000010010110010111111010010010100101010000011000001011001111000100110110001010101010001101101011110000100100',
 '01010110100110000101010101010101000000101000110100100001101110111010011011111010010101100100101010100100001101101100101111000010',
 '00101010011101000000001010101110001000001101010001010111000100001010110110101001100110001010010000111001000010010011100101110000',
 '01101101010010100001001110000101100010000011001110010011010000110110111111101110010000111101011000101010010110110001110110100010',
 '11100010001010011110011110111010011100011101111111101010010001110010001110110100111111011111100100011000001101001011010110011111',
 '01101011111101000010111101011111011010011010000100101101100101010100000010101111001111001010000011000110000110110011010000000101',
 '01111000110111110110001110010100000110011110100010100110111010110111100010001101101001000000100000011001111101110110011100111100',
 '10110110001000001100001011011100110100001011100011011001010110110010110001101111010100010010101011011010101100101111001011000000',
 '11100001100110100111100000001101011010011110100011100011101010010001110111111100110010011100100000011010111011000110011110110100',
 '10010101101111111000001110000000001000011011011011010110111110010100100010010000101101001001100001010111010101011101011110110000',
 '11100101100110011000110101010001010110101100001001101011110111011101101100110111000110001100010100101100100110110110001001101010',
 '00111001111000010001010001011001111000101010110101100001000100001000010000000111111101111100000010111000000100001110000001010111',
 '00000110110110000011111111011110010100001001001011101110111100011001111101010101101100111000011101011010100110110000011111011010',
 '10011000100100110100110101111110001010010110101000100110001010010101001101011101000101110001001000010101100001011101101000110101',
 '10100000011110010110101010111000100001011101000010111000010001001110000111111010101110000110111111110011101000110010100100010111',
 '00110110101111000001000011100111011001100101000010110100001101110010101100010000110011101101110011011001000001010011111100000010',
 '01010101110110101100101010001000011101111001111011100110101101010111110110001011010111001010001100000100011110011011110010111111',
 '11111100110111110010001101010100110110011110101100001011110001000010000010011101111111010111001001111111000110000110000010011100',
 '00000111101101110101000101100011100010111100011001001011000100110001100101011100111101101000111010111110101000100000010110110011',
 '00110000110111111110101111101101101011001011010000001110010001011001100001100010111100001101000010001110101000010100110100000010',
 '01010010100101110011001011101101011100101110110000101011101011100011000010000000110010111111100011011111001100110100010100101011',
 '10010100101001100001111000001101011000010000110111111010110011011110001001101111110110011111001010111011000101000010101001011001',
 '01110111011100011010111010111100110101010000011100100111111101011011101010001100110101011011001110000111011110100000110010110101',
 '00101001101100100010011001011111100000001011001001111100001101001001100001100100111100101011010101101010001110000010101101011010',
 '01000011110100101100011111101000001011101101010001111111111011000110110011110000001100100110110000100111000110111000000110101000',
 '01101111000101010001011010100110110000000001100101100110101000111001111010111001011100110010001011101011110111101101010101000000',
 '10011011110011010000100100100000111001111010000110000001011000011100010111000001111110001001100110101110101100010010000110000010',
 '00011101111001101010011111011001010101101101100001111001100110100011011010001110101111111000000110101000111010001001110110110011',
 '10001011011011101010101010011100011010101100011000010110110101011111110100001100100001010100011011100001001110011110011110110001',
 '11111000101111011010011011011011100001011110111001111010000001000100001011011000000101110010010111000010111000010001111110111001',
 '10101110110010101010101000000111000100000111110100010101000101001000000000011100011000110001010111110100111000011101000110001011',
 '00001101110100011001101011111110000000101000110111111110000111001010111111000100111110110001101101010101010011100100000000000101',
 '01110011010111010111111010011101000010010001000001111110001011000011011110000101110110010100110000011110111111010011000001110010',
 '01010100100110010111100100000011101010110100000110011101010111000000101100101111000110010000001000001010000101011011010000001000',
 '10100001001001101111011001111000110100010011101001111110101100101010001011001110101111010110001010011011101000100111001111010111',
 '00010011101001111010101011101100000000001101100010111010000101001001001111010010011101011111110101011001101001001011000000000000',
 '10111010000001011111011101011001001011010011000001010000100111101100111011000000101000110110101111111110110001001000101010110100',
 '10000110111101101111101101011010011010111100011110101110010111100110100101011111110011000010011110010000011101100101011011100000',
 '00001010011110001000110010000010011000101110101010101100000101000100001110111101011110111111010101110110111001101011111100010011',
 '11100010000110001101001011010011010100101110110111011011010010101000110101011110010101000000110000000101101101010101010110111011',
 '01000010010010101100101010101001111010010000000100001001010101011110101111000010111111111100001010100001101111110000100111011100',
 '11001010010011000111101010101010100111011000010000011101011000000001010001000001000101000111101110000010011010011011100101011010',
 '01000101011100101011110011011000010111010001010001010110101001101100101000000111110100001000010010001000111111000100110101001010',
 '00101111111110010101111110001010011110011110000001001001001001101011001101000100100101000000001110110111110111101110100000010100',
 '00011100101111100111000000111110010111010111110100101100000000100111011110001001001000001010111001111111101110010000101101011111',
 '00001011011100100010100110101000010100011001101000001100110001001110111010100001111011100001000011100111010101011011101001000010',
 '00010110001110011010010011000101101011000100100100000100011000001101100101111011101100010101010101101110010110000010111010010001',
 '01101001111111001011101011111001100010000001110010100100110110110001110010110111011111111000001000000011100101001101101000100110',
 '10101101000001011111000010011110111011111100011110011110010110001000011101110000101010011001000000101111001000000000101111100010',
 '00001001011110011001101100101000000011000011011001101001010101001101011111100111101110110111111111100010111001111001000010100101',
 '11010000000100010101000011110001001001011101011001001000000001011000011110010100101111100111010011111111111001001001010110001001',
 '00000010100100001110011111010101101010100100011101000000011011110010101101001111111001001010010111100010000011110000001010101000',
 '01110111011011010000010111111110101001101101110100111011001000001001011111010010111000011101100010111100010110100100101001010001',
 '11011110010111101011111101110111101111000001101010001101100110111101101010000000111000011100000011010111110010100111011111101001',
 '10011100000010110100100111010100101011101010001011100000101111000010000100000011101001001101110100110110010011101001101111100010',
 '11100011011011001100000011110111110000100111110001100010100100110011010111101111000100100101110110101101011010011001111000111000',
 '01001001101001101001000001101110000011001010001000111100101111110010100001101110110001101000011001000100100001011101101000100001',
 '01010101011011111000000110000100001001010100101101000110001101101101100110110110101011111011011001001101110111101110101000101110',
 '00011101101110000111001011101100111001101011001011110001000110001100111111101111010011001100010011011011100100101100010110100010',
 '00110100001010001110011011110011100111111010010100101111000111000011111110111111001100100010101011010010111010101101110001111100',
 '11100111001001111100100000001000111000110111000111000001001010011000011100101110011101111001001010111101000101110010000110111101',
 '11101011010000100011100110000001010000011111111110110011110010011101000101101010010110101010101010001110100011001011110000100000',
 '01101001110110000100110100110001111101111011011110010111110100111111100111100011111011110101100100011110100100010000101011100000',
 '00011111111111101011001001110011011100000011011000000001111010110100111101000100001111000000010001111000101101100001110001101101',
 '10101100010101010100000100100011100101110000100101000100100101100110111001010111000100110111101001101011000110010000010011110001',
 '10101000101100100111010010000111101110001110001001010001100100000010101110110100111111000011011011010010100001100111111100001100',
 '01101010111001111001001010100000011010010100001010001101001010000010000110101011011010111110011100110110001110000101111101100110',
 '10100000010000000100001001001000010101011101000001000000111100001001010001000011000111100100000110110101011101000000001010010100',
 '10001111101100100001101101111100001100010111100000110100011011001010011100100100010000000011100010111111100110010000110110011011',
 '10001110000111110101101010010000011111001100010010011011000111011100011110101010101011010000000111000000111101010011110010101101',
 '11000001001001100100011010010001000111100111110010110101101101101100011101010110100010010001111010100000101001111101101010001010',
 '00010101110100110001010011010110001001011001011100101001100111100000101011000001011001111111000011010001100111111010000110110010',
 '11011101111000001000000111000010100100011001000100111011011010010010010100110001011011100010001000000001010001010100010001110100',
 '00101110010111010011011001000111100110100101001100110001111011100011001001111111000010110010110000111010001101011001001011100011',
 '11110101101111101111010101111010100011011011011101000111111001001011011000111100100100111100110110111111000100101101000011110100',
 '10000101000100000110100010011111100101100110111111110011111101011000110010011000011101111001000010111000101100001100000010100110',
 '10101000101111111100101000000101101011101110000000001001001010001000100010101000101001101111000000100010010101011110111001011000',
 '00010001100110101010011010011010001101100100110111100001100001101001001011000110110000000000111011010101101011011010111011100111',
 '10000111001010011001100001110111100011000011010101111100000100010100110001110010010101001111100101011100111101111011010000011001',
 '01101100110010011011100111011111100100010101001011000001001000110000000011100100110110000010010011100101001100110011101100010111',
 '10101101000100110100000101111110111110010111101000010110001110011011100011000101100110001000011111111100010011111011011101001011',
 '10101010010110111011101111110101101111011100001011001000111101010000111111011100100001110001100010111100001100001000101110110110',
 '10100001001010001101010001110111100011000111101011110101111111011001010011010000111100100011101110010010100010010110110100101111',
 '00011110001000010100001100000001100101001010011100101110001101011111000010000100111001000011000001111010110000001100111110000000',
 '11000001111110010001011110011001001010101000100101110110010010011001011001111001011010000101011110001110000111000100001100101011',
 '11100011111011111111101111010011010111100111011011011001101111111001101000111100101001101100011011000001010010101000001110100100',
 '11011010011000111000010010101100111001111010000110100111100100110101000010111001101101110011011100000010000010011000110100001100',
 '10101000100011010101111110000010010000101000010100011010101100100001001110010100011111110111001101010011101110111111100000001100',
 '11111010100100111101110001011000010000110000000100101011011101011101001010101110101011010010100110110001000000101111110101000111',
 '00010011010101110000110110110001001000001110111000100011001101000010100010011010111011111110101000111101100010000010111000001101',
 '11001000010110001010010011000010111011110111001010100010101101000110101101011100010101010011010110000000010101011000011001111001',
 '01001110100110110000000101100011110100011010010001110011001111001000100111100101111010010001111111010110111101111101111111101001',
 '01011010001111011011100011000001101000101111111101001101011010101000001101111110100100011100011011011001010101110110001000100010',
 '00100000111001010101011001010010011001011101100010011110001000101001111010001000100101000001110100001110010101111110110011101010',
 '00110111101101010000111001001000010100111101010001001110111100111111110110011001010101110000100101010011011011110100010110111101',
 '00110001100111010110100000111010100110110111000010111110101010010110110100100010000101001100100110000110010001111110000010110001',
 '00111010101110100000101000010110110011100011100100100100010010100001000101000101110111100100001000100001100000000100101000100110',
 '01101010001011100000010111101111000001010001001101101111010000111010000001010111111111011110101100101110010000010011101100100001',
 '11101101010110111100110000110110110011110101100011110100111011010101001101001111100101100110110001101001011011110011111000011100',
 '11010011000110101000011101011001110110000010011111000100111011101001111110010000000110110111010010101011000010001001110010011101',
 '00111110110010101111000011010010011001000110111010011101110000010100100000111110011101010010110101010010011010001000101000011110']